Part 1 of practice activities for 7.2:

In [2]:
import pandas as pd
ames = pd.read_csv('AmesHousing.csv')
ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [3]:
# Get rid of columns with mostly NaN values
good_cols = ames.isna().sum() < 100
ames = ames.loc[:,good_cols]

# Drop other NAs
ames = ames.dropna()

In [5]:
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np
X = ames.drop(["SalePrice", "Order", "PID"], axis = 1)
y = ames["SalePrice"]


ct = ColumnTransformer(
  [
    ("dummify",
    OneHotEncoder(sparse_output = False, handle_unknown='ignore'),
    make_column_selector(dtype_include=object)),
    ("standardize",
    StandardScaler(),
    make_column_selector(dtype_include=np.number))
  ],
  remainder = "passthrough"
)

lr_pipeline_1 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

In [7]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr_pipeline_1, X, y, cv = 5, scoring = 'r2')

array([0.89727873, 0.91038253, 0.78900365, 0.77208628, 0.9006982 ])

In [12]:
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
import numpy as np
X = ames.drop(["SalePrice", "Order", "PID"], axis = 1)
y = ames["SalePrice"]


ct = ColumnTransformer(
  [
    ("dummify",
    OneHotEncoder(sparse_output = False, handle_unknown='ignore'),
    make_column_selector(dtype_include=object)),
    ("standardize",
    StandardScaler(),
    make_column_selector(dtype_include=np.number))
  ],
  remainder = "passthrough"
)

ridge_pipeline_1 = Pipeline(
  [("preprocessing", ct),
  ("ridge_regression", Ridge(alpha = 1))]
)

In [14]:
from sklearn.model_selection import cross_val_score
cross_val_score(ridge_pipeline_1, X, y, cv = 5, scoring = 'r2')

array([0.89815807, 0.91744024, 0.79493606, 0.78522563, 0.91389818])

Part 2:

In [51]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'ridge_regression__alpha': [0.001, 0.01, 0.1, 1, 10]
}

In [52]:
ridge_pipeline_tuned = Pipeline(
  [("preprocessing", ct),
  ("ridge_regression", Ridge(alpha = 1))]
)

In [53]:
grid_search_ridge = GridSearchCV(
    estimator=ridge_pipeline_tuned,
    param_grid=param_grid,
    cv=5,
    scoring='r2'
)

grid_search_ridge.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('dummify',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x783254265220>),
                                                                        ('standardize',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x783254265790>)])),
                                       ('ridge_regression', Ridge(alpha=1))]),
             param_grid={'ridge_regression__alpha': [0.001, 0.01, 0.1, 1, 10]},
             scoring='r2')

In [54]:
print("Best parameters found: ", grid_search_ridge.best_params_)
print("Best R-squared score: ", grid_search_ridge.best_score_)

Best parameters found:  {'ridge_regression__alpha': 10}
Best R-squared score:  0.8642722110166747


Part 3: Lasso pipeline

In [47]:
param_grid = {
    'lasso_regression__alpha': [0.001, 0.01, 0.1, 1, 10]
}

In [48]:
from sklearn.linear_model import Lasso

lasso_pipeline_tuned = Pipeline(
  [("preprocessing", ct),
  ("lasso_regression", Lasso(alpha = 1))])

In [49]:
grid_search_lasso = GridSearchCV(
    estimator=lasso_pipeline_tuned,
    param_grid=param_grid,
    cv=5,
    scoring='r2'
)

grid_search_lasso.fit(X, y)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.109e+11, tolerance: 1.348e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.466e+11, tolerance: 1.474e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.894e+11, tolerance: 1.463e

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('dummify',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x783254265220>),
                                                                        ('standardize',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x783254265790>)])),
                                       ('lasso_regression', Lasso(alpha=1))]),
             param_grid={'lasso_regression__alpha': [0.001, 0.01, 0.1, 1, 10]},
             scoring='r2')

In [50]:
print("Best parameters found: ", grid_search_lasso.best_params_)
print("Best R-squared score: ", grid_search_lasso.best_score_)

Best parameters found:  {'lasso_regression__alpha': 10}
Best R-squared score:  0.860632243732114


Part 3: Elastic Net

In [55]:
param_grid = {
    'elastic_net__alpha': [0.001, 0.01, 0.1, 1, 10]
}

In [65]:
from sklearn.linear_model import ElasticNet
elastic_net_pipeline_tuned = Pipeline(
  [("preprocessing", ct),
  ("elastic_net", ElasticNet(alpha = 1))])

In [66]:
grid_search_net_elastic = GridSearchCV(
    estimator=elastic_net_pipeline_tuned,
    param_grid=param_grid,
    cv=5,
    scoring='r2'
)

grid_search_net_elastic.fit(X, y)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.191e+11, tolerance: 1.348e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.975e+11, tolerance: 1.474e+09
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.005e+11, tolerance: 1.463e

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('dummify',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x783254265220>),
                                                                        ('standardize',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x783254265790>)])),
                                       ('elastic_net', ElasticNet(alpha=1))]),
             param_grid={'elastic_net__alpha': [0.001, 0.01, 0.1, 1, 10]},
             scoring='r2')

In [67]:
print("Best parameters found: ", grid_search_net_elastic.best_params_)
print("Best R-squared score: ", grid_search_net_elastic.best_score_)

Best parameters found:  {'elastic_net__alpha': 0.01}
Best R-squared score:  0.8642678883652006
